# Movie details scraping from TMDB (https://www.themoviedb.org/movie)
### columns scraped:
- movie title
- movie overview (description)
- movie rating
- movie genre

### Total data scraped: 8000 rows, 4 columns (first 400 pages)

### Project Background: 
Many show streaming platforms sort their shows and movies into various categories and genres, to give viewers a gist of what to expect when starting a new series or movie. Furthermore, this catalogued system gives viewers a comprehensive and personalised experience from the convenience of their device. However, the downside of this large influx of content might make it difficult for streaming services to allocate time and resources for such manual tagging. For example, Netflix employs the help of ‘Netflix taggers’ who watch the show for them and tags the various content. 

### Project Objective: 
- This project aims to implement an Automated Genre Tagging system for the movie website that accurately categorises and organises movies using relevant keywords and categories, without requiring manual input from staff. 
- Our sub objective is to create a personalised Movie Recommendation system based on user input and preferences. This feature will enable users to discover and watch movies they’re likely to be interested in, while reducing the need to scroll through irrelevant titles, giving them a seamless, personalised experience.

### Scraping process:
1. Scraped all the href links of each movie which leads to their individual detailed movie page containing their overview and genre. e.g. https://www.themoviedb.org/movie/536554
2. Performed pagination which allowed me to scrape all 4000 pages of movies' detailed pages (URL). Each page contains 20 movies. The list containing all 8000 movie detail page links is called full_list.
3. Finally start web scraping the movie details: title, rating, genres and overview of each movie by downloading each link's html (movie) and scraping each movie html using functions.
4. Loop it 8000 times to get every single movie's details and store them into lists (title_list, tag_genre_list, overview_list, ratings_list)
5. Place the lists into a dictionary
6. Convert dictionary into dataframe
7. Write the dataframe into a csv file for text pre-processing.

#### Adjustments made: 
In the proposal, it was stated that each group member would scrape movies of the genres that were allocated to them. However for my website, there is no way for me to scrape only a certain genre as the website's genre filter will be reseted everytime the website is refreshed. Therefore, I scrapped more movies than my group members and will then filter out and use only my allocated genres in my Text Pre-processing notebook's data cleaning step. Other than this genre matter, our scraped movie data format is actually the same, we have the same order and number of columns, thus we do not need an extra merging dataset notebook.

In [1]:
# Load the packages
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [33]:
# Defining the url of the site
base_site = "https://www.themoviedb.org/movie"
base_url = 'http://themoviedb.org'
# Making a get request
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
response = requests.get(base_site, headers=headers)

#     print(response.content.decode())

In [34]:
response.status_code

200

##### Extracting the HTML

In [6]:
html = response.content

##### Checking that the reply is indeed an HTML code by inspecting the first 100 symbols

In [7]:
html[:100]

b'<!DOCTYPE html>\n<html lang="en" class="no-js">\n  <head>\n    <title>Popular Movies &#8212; The Movie '

##### Defining a variable 'soup' to store the html content

In [8]:
soup = BeautifulSoup(html, 'html.parser')

##### Create function to download page 

In [9]:
def download_page(url):
    # In this case, we are going to give request.get function headers to avoid the Status code Error 403

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    # we are going to raise exception here if status code gives any value other than 200.
    if not response.ok:
        raise Exception ("Failed to request the data. Status Code:- {}".format(response.status_code))
    else:
        html = response.content
        soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
        return soup

##### Defining a variable 'soup' to store the html content

In [10]:
soup = download_page(base_site)

#### Testing 

#### Scrape title of the first movie

In [11]:
title = soup.find_all('div', {'class': 'card style_1'})[0].h2.text
title

'Puss in Boots: The Last Wish'

#### Scrape rating of the first movie

In [12]:
rating = [soup.find_all('div', {'class': 'user_score_chart'})[0]['data-percent']]
rating

['86.0']

### Scraping detailed page's contents

#### In order to navigate to the details page, the user needs to click on the movie image to find out more about the movie, including its description which we are trying to scrape. It turns out each movie has its unique identifier (e.g. 315162) and lives within <a> tags in images. Let's see what is in the first image tag in the website

In [15]:
page_wrapper = soup.find('div', class_='page_wrapper')
img_tags = page_wrapper.find_all('a', class_='image')
img_tags[0]

<a class="image" href="/movie/315162" title="Puss in Boots: The Last Wish">
<img alt="" class="poster" loading="lazy" src="/t/p/w220_and_h330_face/kuf6dutpsT0vSVehic3EZIqkOBt.jpg" srcset="/t/p/w220_and_h330_face/kuf6dutpsT0vSVehic3EZIqkOBt.jpg 1x, /t/p/w440_and_h660_face/kuf6dutpsT0vSVehic3EZIqkOBt.jpg 2x"/>
</a>

##### Scraping each href that can be combined with the base_url 'https://www.themoviedb.org' to obtain the URL to each movie detail page

In [16]:
#To get the value of <a href=XXX>
a_image_tags = page_wrapper.find_all('a', class_='image')
href_value = a_image_tags[0]['href']
href_value

'/movie/315162'

##### To concatate base url with the attribute href of a tags

In [18]:
details_page = base_url + href_value
print(details_page)

http://themoviedb.org/movie/315162


##### Make a for loop to test the code for page 1

##### To get a list of details page on the same page

In [20]:
base_url = 'https://www.themoviedb.org'
content_tags = page_wrapper.find_all('div',class_="content")
a_tags_for_details = [content_tag.find('a') for content_tag in content_tags]
detail_pages = [base_url+a_tag_for_details['href'] for a_tag_for_details in a_tags_for_details ]
print("the total number of detailed pages {}".format(len(detail_pages)))
detail_pages

the total number of detailed pages 20


['https://www.themoviedb.org/movie/315162',
 'https://www.themoviedb.org/movie/536554',
 'https://www.themoviedb.org/movie/76600',
 'https://www.themoviedb.org/movie/846433',
 'https://www.themoviedb.org/movie/653851',
 'https://www.themoviedb.org/movie/899112',
 'https://www.themoviedb.org/movie/736526',
 'https://www.themoviedb.org/movie/436270',
 'https://www.themoviedb.org/movie/19995',
 'https://www.themoviedb.org/movie/505642',
 'https://www.themoviedb.org/movie/740952',
 'https://www.themoviedb.org/movie/661374',
 'https://www.themoviedb.org/movie/411',
 'https://www.themoviedb.org/movie/1041513',
 'https://www.themoviedb.org/movie/683328',
 'https://www.themoviedb.org/movie/724495',
 'https://www.themoviedb.org/movie/668482',
 'https://www.themoviedb.org/movie/934287',
 'https://www.themoviedb.org/movie/676547',
 'https://www.themoviedb.org/movie/81774']

append all movies into a list 

##### Let's try it with pagination this time

In [22]:
#scrape all the detail pages together with pagination 
pag_list = []
full_movies_list = []

#Scrape all 400 pages of popular movies
for i in range(1,401):
    pag = f'https://www.themoviedb.org/movie?page={i}'
    pag_list.append(pag)    
    soup = download_page(pag)
#     print(soup)
    # To get a list of details page on the same pages
    base_url = 'https://www.themoviedb.org'
    page_wrapper = soup.find('div', class_='page_wrapper')
    content_tags = page_wrapper.find_all('div',class_="content")
    a_tags_for_details = [content_tag.find('a') for content_tag in content_tags]
    detail_pages = [base_url+a_tag_for_details['href'] for a_tag_for_details in a_tags_for_details]
    print("the total number of detailed pages on page {} is {}".format(i,len(detail_pages)))
    print(detail_pages)
    print('\n')
    full_movies_list.append(detail_pages)
# print(pag_list)

the total number of detailed pages on page 1 is 20
['https://www.themoviedb.org/movie/315162', 'https://www.themoviedb.org/movie/536554', 'https://www.themoviedb.org/movie/76600', 'https://www.themoviedb.org/movie/846433', 'https://www.themoviedb.org/movie/653851', 'https://www.themoviedb.org/movie/899112', 'https://www.themoviedb.org/movie/736526', 'https://www.themoviedb.org/movie/436270', 'https://www.themoviedb.org/movie/19995', 'https://www.themoviedb.org/movie/505642', 'https://www.themoviedb.org/movie/740952', 'https://www.themoviedb.org/movie/661374', 'https://www.themoviedb.org/movie/411', 'https://www.themoviedb.org/movie/1041513', 'https://www.themoviedb.org/movie/683328', 'https://www.themoviedb.org/movie/724495', 'https://www.themoviedb.org/movie/668482', 'https://www.themoviedb.org/movie/934287', 'https://www.themoviedb.org/movie/676547', 'https://www.themoviedb.org/movie/81774']


the total number of detailed pages on page 2 is 20
['https://www.themoviedb.org/movie/87312

the total number of detailed pages on page 10 is 20
['https://www.themoviedb.org/movie/799379', 'https://www.themoviedb.org/movie/429473', 'https://www.themoviedb.org/movie/639933', 'https://www.themoviedb.org/movie/752623', 'https://www.themoviedb.org/movie/829560', 'https://www.themoviedb.org/movie/676841', 'https://www.themoviedb.org/movie/497828', 'https://www.themoviedb.org/movie/579974', 'https://www.themoviedb.org/movie/32516', 'https://www.themoviedb.org/movie/846778', 'https://www.themoviedb.org/movie/580489', 'https://www.themoviedb.org/movie/782054', 'https://www.themoviedb.org/movie/883987', 'https://www.themoviedb.org/movie/718789', 'https://www.themoviedb.org/movie/284329', 'https://www.themoviedb.org/movie/566466', 'https://www.themoviedb.org/movie/879538', 'https://www.themoviedb.org/movie/185665', 'https://www.themoviedb.org/movie/928015', 'https://www.themoviedb.org/movie/767']


the total number of detailed pages on page 11 is 20
['https://www.themoviedb.org/movie/10

the total number of detailed pages on page 20 is 20
['https://www.themoviedb.org/movie/214756', 'https://www.themoviedb.org/movie/166428', 'https://www.themoviedb.org/movie/1050040', 'https://www.themoviedb.org/movie/619730', 'https://www.themoviedb.org/movie/637649', 'https://www.themoviedb.org/movie/713704', 'https://www.themoviedb.org/movie/701952', 'https://www.themoviedb.org/movie/812488', 'https://www.themoviedb.org/movie/951368', 'https://www.themoviedb.org/movie/1054513', 'https://www.themoviedb.org/movie/762975', 'https://www.themoviedb.org/movie/945675', 'https://www.themoviedb.org/movie/913290', 'https://www.themoviedb.org/movie/23823', 'https://www.themoviedb.org/movie/49013', 'https://www.themoviedb.org/movie/20982', 'https://www.themoviedb.org/movie/341174', 'https://www.themoviedb.org/movie/14836', 'https://www.themoviedb.org/movie/269149', 'https://www.themoviedb.org/movie/863']


the total number of detailed pages on page 21 is 20
['https://www.themoviedb.org/movie/589

the total number of detailed pages on page 30 is 20
['https://www.themoviedb.org/movie/475557', 'https://www.themoviedb.org/movie/912598', 'https://www.themoviedb.org/movie/337339', 'https://www.themoviedb.org/movie/862551', 'https://www.themoviedb.org/movie/47971', 'https://www.themoviedb.org/movie/301502', 'https://www.themoviedb.org/movie/381284', 'https://www.themoviedb.org/movie/628878', 'https://www.themoviedb.org/movie/245891', 'https://www.themoviedb.org/movie/615469', 'https://www.themoviedb.org/movie/10020', 'https://www.themoviedb.org/movie/603', 'https://www.themoviedb.org/movie/899082', 'https://www.themoviedb.org/movie/11224', 'https://www.themoviedb.org/movie/666949', 'https://www.themoviedb.org/movie/102382', 'https://www.themoviedb.org/movie/221912', 'https://www.themoviedb.org/movie/588108', 'https://www.themoviedb.org/movie/632727', 'https://www.themoviedb.org/movie/764835']


the total number of detailed pages on page 31 is 20
['https://www.themoviedb.org/movie/7037

the total number of detailed pages on page 40 is 20
['https://www.themoviedb.org/movie/102651', 'https://www.themoviedb.org/movie/218746', 'https://www.themoviedb.org/movie/640', 'https://www.themoviedb.org/movie/539885', 'https://www.themoviedb.org/movie/496450', 'https://www.themoviedb.org/movie/50546', 'https://www.themoviedb.org/movie/284052', 'https://www.themoviedb.org/movie/49026', 'https://www.themoviedb.org/movie/351819', 'https://www.themoviedb.org/movie/420809', 'https://www.themoviedb.org/movie/728366', 'https://www.themoviedb.org/movie/1010819', 'https://www.themoviedb.org/movie/408270', 'https://www.themoviedb.org/movie/800939', 'https://www.themoviedb.org/movie/888097', 'https://www.themoviedb.org/movie/397567', 'https://www.themoviedb.org/movie/489', 'https://www.themoviedb.org/movie/106909', 'https://www.themoviedb.org/movie/447404', 'https://www.themoviedb.org/movie/22894']


the total number of detailed pages on page 41 is 20
['https://www.themoviedb.org/movie/634528

...

the total number of detailed pages on page 359 is 20
['https://www.themoviedb.org/movie/729454', 'https://www.themoviedb.org/movie/13765', 'https://www.themoviedb.org/movie/9576', 'https://www.themoviedb.org/movie/532444', 'https://www.themoviedb.org/movie/365045', 'https://www.themoviedb.org/movie/334524', 'https://www.themoviedb.org/movie/986259', 'https://www.themoviedb.org/movie/413126', 'https://www.themoviedb.org/movie/74018', 'https://www.themoviedb.org/movie/453191', 'https://www.themoviedb.org/movie/1588', 'https://www.themoviedb.org/movie/14979', 'https://www.themoviedb.org/movie/18011', 'https://www.themoviedb.org/movie/13333', 'https://www.themoviedb.org/movie/339547', 'https://www.themoviedb.org/movie/146239', 'https://www.themoviedb.org/movie/16304', 'https://www.themoviedb.org/movie/682110', 'https://www.themoviedb.org/movie/25538', 'https://www.themoviedb.org/movie/47166']


the total number of detailed pages on page 360 is 20
['https://www.themoviedb.org/movie/3110', '

the total number of detailed pages on page 369 is 20
['https://www.themoviedb.org/movie/15809', 'https://www.themoviedb.org/movie/821', 'https://www.themoviedb.org/movie/44040', 'https://www.themoviedb.org/movie/288418', 'https://www.themoviedb.org/movie/826753', 'https://www.themoviedb.org/movie/178809', 'https://www.themoviedb.org/movie/70841', 'https://www.themoviedb.org/movie/818809', 'https://www.themoviedb.org/movie/801', 'https://www.themoviedb.org/movie/188813', 'https://www.themoviedb.org/movie/449756', 'https://www.themoviedb.org/movie/17457', 'https://www.themoviedb.org/movie/9474', 'https://www.themoviedb.org/movie/313922', 'https://www.themoviedb.org/movie/14506', 'https://www.themoviedb.org/movie/11010', 'https://www.themoviedb.org/movie/489326', 'https://www.themoviedb.org/movie/641662', 'https://www.themoviedb.org/movie/9624', 'https://www.themoviedb.org/movie/11336']


the total number of detailed pages on page 370 is 20
['https://www.themoviedb.org/movie/283552', 'htt

the total number of detailed pages on page 379 is 20
['https://www.themoviedb.org/movie/411728', 'https://www.themoviedb.org/movie/98948', 'https://www.themoviedb.org/movie/11471', 'https://www.themoviedb.org/movie/785853', 'https://www.themoviedb.org/movie/339987', 'https://www.themoviedb.org/movie/821494', 'https://www.themoviedb.org/movie/10122', 'https://www.themoviedb.org/movie/48492', 'https://www.themoviedb.org/movie/40084', 'https://www.themoviedb.org/movie/10546', 'https://www.themoviedb.org/movie/74643', 'https://www.themoviedb.org/movie/559638', 'https://www.themoviedb.org/movie/2011', 'https://www.themoviedb.org/movie/185668', 'https://www.themoviedb.org/movie/40394', 'https://www.themoviedb.org/movie/57157', 'https://www.themoviedb.org/movie/14787', 'https://www.themoviedb.org/movie/11543', 'https://www.themoviedb.org/movie/1957', 'https://www.themoviedb.org/movie/12182']


the total number of detailed pages on page 380 is 20
['https://www.themoviedb.org/movie/38778', 'htt

the total number of detailed pages on page 389 is 20
['https://www.themoviedb.org/movie/11536', 'https://www.themoviedb.org/movie/13920', 'https://www.themoviedb.org/movie/1070893', 'https://www.themoviedb.org/movie/892429', 'https://www.themoviedb.org/movie/444902', 'https://www.themoviedb.org/movie/203696', 'https://www.themoviedb.org/movie/724108', 'https://www.themoviedb.org/movie/36073', 'https://www.themoviedb.org/movie/664332', 'https://www.themoviedb.org/movie/39264', 'https://www.themoviedb.org/movie/177945', 'https://www.themoviedb.org/movie/105203', 'https://www.themoviedb.org/movie/12103', 'https://www.themoviedb.org/movie/418078', 'https://www.themoviedb.org/movie/18835', 'https://www.themoviedb.org/movie/4413', 'https://www.themoviedb.org/movie/1863', 'https://www.themoviedb.org/movie/12500', 'https://www.themoviedb.org/movie/269242', 'https://www.themoviedb.org/movie/11148']


the total number of detailed pages on page 390 is 20
['https://www.themoviedb.org/movie/160588'

the total number of detailed pages on page 399 is 20
['https://www.themoviedb.org/movie/281', 'https://www.themoviedb.org/movie/491472', 'https://www.themoviedb.org/movie/519337', 'https://www.themoviedb.org/movie/828149', 'https://www.themoviedb.org/movie/12498', 'https://www.themoviedb.org/movie/880009', 'https://www.themoviedb.org/movie/484437', 'https://www.themoviedb.org/movie/430354', 'https://www.themoviedb.org/movie/18781', 'https://www.themoviedb.org/movie/593680', 'https://www.themoviedb.org/movie/33380', 'https://www.themoviedb.org/movie/14113', 'https://www.themoviedb.org/movie/13991', 'https://www.themoviedb.org/movie/528085', 'https://www.themoviedb.org/movie/458109', 'https://www.themoviedb.org/movie/3536', 'https://www.themoviedb.org/movie/40211', 'https://www.themoviedb.org/movie/2294', 'https://www.themoviedb.org/movie/23127', 'https://www.themoviedb.org/movie/13815']


the total number of detailed pages on page 400 is 20
['https://www.themoviedb.org/movie/474354', 'h

#### Combine lists together into a huge list

In [23]:
full_list = []
for list in full_movies_list:
    for link in list:
        full_list.append(link)
full_list

['https://www.themoviedb.org/movie/315162',
 'https://www.themoviedb.org/movie/536554',
 'https://www.themoviedb.org/movie/76600',
 'https://www.themoviedb.org/movie/846433',
 'https://www.themoviedb.org/movie/653851',
 'https://www.themoviedb.org/movie/899112',
 'https://www.themoviedb.org/movie/736526',
 'https://www.themoviedb.org/movie/436270',
 'https://www.themoviedb.org/movie/19995',
 'https://www.themoviedb.org/movie/505642',
 'https://www.themoviedb.org/movie/740952',
 'https://www.themoviedb.org/movie/661374',
 'https://www.themoviedb.org/movie/411',
 'https://www.themoviedb.org/movie/1041513',
 'https://www.themoviedb.org/movie/683328',
 'https://www.themoviedb.org/movie/724495',
 'https://www.themoviedb.org/movie/668482',
 'https://www.themoviedb.org/movie/934287',
 'https://www.themoviedb.org/movie/676547',
 'https://www.themoviedb.org/movie/81774',
 'https://www.themoviedb.org/movie/873126',
 'https://www.themoviedb.org/movie/1015963',
 'https://www.themoviedb.org/movie/9

#####  Function to obtain genre from soup

In [24]:
def get_genre():
    try:
        tag_genre = soup.find('span', {"class": "genres"})
        tag_genre_list = tag_genre.find_all('a')
        
        #create genre list to collect all genres as there may be more than 1 genre in a movie
        check_genre =[]
        
        for tag in tag_genre_list:
            check_genre.append(tag.text)
        return check_genre
    
    except AttributeError:
        print('genre not found')

#####  Function to obtain overview from soup

In [26]:
def get_overview():
    try: 
        overview=soup.find("div", {"class":"overview"}).p
        return overview.text
    
    except AttributeError:
        print('overview not found')

#####  Function to obtain title from soup

In [27]:
def get_title():
    try:
#         link = 'https://www.themoviedb.org/movie/1045944'
#         soup = download_page(link)
        # print(soup)
        title = soup.find('section', attrs={"class": "header poster"}).a.text
        return title

    except AttributeError:
        print('title not found')

##### Using all functions together to loop through each of the movie URL in full_list to obtain each movie's information

In [28]:
title_list = []
ratings_list = []
tag_genre_list = []
overview_list = []
counter = 1

for link in full_list:
    
    soup = download_page(link)
    
    # extract titles of all the movies
    title = get_title()
    title_list.append(title)
    
    # extract rating of all the movies
    ratings = soup.find_all('div', {'class': 'user_score_chart'})[0]['data-percent']
    ratings_list.append(ratings)
    
    # extract genres of all the movies
    tag_genre = get_genre()
    tag_genre_list.append(tag_genre)
    
    # extract overview of all the movies
    overview = get_overview()
    overview_list.append(overview)
    
    #Create a counter to see progress 
    print('printing page', counter)
    counter+=1
    
print('titles:', title_list )
print('\n')
print('ratings:', ratings_list)
print('\n')
print('genres:',tag_genre_list)
print('\n')
print('overviews:', overview_list)



printing page 1
printing page 2
printing page 3
printing page 4
printing page 5
printing page 6
printing page 7
printing page 8
printing page 9
printing page 10
printing page 11
printing page 12
printing page 13
printing page 14
printing page 15
printing page 16
printing page 17
printing page 18
printing page 19
printing page 20
printing page 21
printing page 22
printing page 23
printing page 24
printing page 25
printing page 26
printing page 27
printing page 28
printing page 29
printing page 30
printing page 31
printing page 32
printing page 33
printing page 34
printing page 35
printing page 36
printing page 37
printing page 38
printing page 39
printing page 40
printing page 41
printing page 42
printing page 43
printing page 44
printing page 45
printing page 46
printing page 47
printing page 48
printing page 49
printing page 50
printing page 51
printing page 52
printing page 53
printing page 54
printing page 55
printing page 56
printing page 57
printing page 58
printing page 59
printi

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


printing page 373
printing page 374
printing page 375
printing page 376
printing page 377
printing page 378
printing page 379
printing page 380
printing page 381
printing page 382
printing page 383
printing page 384
printing page 385
printing page 386
printing page 387
printing page 388
printing page 389
printing page 390
printing page 391
printing page 392
printing page 393
printing page 394
printing page 395
printing page 396
printing page 397
printing page 398
printing page 399
printing page 400
printing page 401
printing page 402
printing page 403
title not found
printing page 404
title not found
printing page 405
printing page 406
printing page 407
printing page 408
printing page 409
printing page 410
printing page 411
printing page 412
printing page 413
printing page 414
printing page 415
printing page 416
printing page 417
printing page 418
printing page 419
printing page 420
printing page 421
printing page 422
printing page 423
printing page 424
printing page 425
printing page 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


printing page 497
printing page 498
printing page 499
printing page 500
printing page 501
printing page 502
printing page 503
title not found
printing page 504
printing page 505
printing page 506
printing page 507
printing page 508
printing page 509
printing page 510
printing page 511
printing page 512
printing page 513
printing page 514
printing page 515
printing page 516
printing page 517
printing page 518
printing page 519
printing page 520
title not found
printing page 521
printing page 522
title not found
printing page 523
printing page 524
printing page 525
printing page 526
printing page 527
printing page 528
printing page 529
printing page 530
printing page 531
printing page 532
printing page 533
printing page 534
printing page 535
printing page 536
printing page 537
printing page 538
printing page 539
printing page 540
printing page 541
printing page 542
printing page 543
printing page 544
printing page 545
printing page 546
printing page 547
printing page 548
printing page 54

...

printing page 6890
printing page 6891
printing page 6892
printing page 6893
printing page 6894
printing page 6895
printing page 6896
printing page 6897
printing page 6898
printing page 6899
printing page 6900
printing page 6901
printing page 6902
printing page 6903
printing page 6904
printing page 6905
printing page 6906
printing page 6907
printing page 6908
printing page 6909
printing page 6910
printing page 6911
printing page 6912
printing page 6913
printing page 6914
printing page 6915
printing page 6916
printing page 6917
printing page 6918
printing page 6919
printing page 6920
printing page 6921
printing page 6922
printing page 6923
printing page 6924
title not found
printing page 6925
printing page 6926
printing page 6927
printing page 6928
printing page 6929
printing page 6930
printing page 6931
printing page 6932
printing page 6933
printing page 6934
printing page 6935
printing page 6936
printing page 6937
printing page 6938
printing page 6939
printing page 6940
printing page 6

printing page 7307
printing page 7308
printing page 7309
printing page 7310
printing page 7311
printing page 7312
printing page 7313
printing page 7314
printing page 7315
printing page 7316
printing page 7317
printing page 7318
printing page 7319
printing page 7320
printing page 7321
printing page 7322
printing page 7323
printing page 7324
printing page 7325
printing page 7326
printing page 7327
printing page 7328
printing page 7329
printing page 7330
printing page 7331
printing page 7332
printing page 7333
printing page 7334
printing page 7335
title not found
printing page 7336
printing page 7337
printing page 7338
printing page 7339
printing page 7340
printing page 7341
printing page 7342
printing page 7343
printing page 7344
title not found
printing page 7345
printing page 7346
printing page 7347
printing page 7348
printing page 7349
printing page 7350
printing page 7351
printing page 7352
title not found
printing page 7353
printing page 7354
printing page 7355
printing page 7356
pr

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


printing page 7532
printing page 7533
printing page 7534
printing page 7535
printing page 7536
printing page 7537
printing page 7538
printing page 7539
printing page 7540
printing page 7541
printing page 7542
printing page 7543
printing page 7544
printing page 7545
printing page 7546
printing page 7547
printing page 7548
printing page 7549
printing page 7550
printing page 7551
title not found
printing page 7552
printing page 7553
title not found
printing page 7554
printing page 7555
printing page 7556
printing page 7557
printing page 7558
printing page 7559
printing page 7560
printing page 7561
printing page 7562
printing page 7563
printing page 7564
printing page 7565
printing page 7566
printing page 7567
printing page 7568
printing page 7569
printing page 7570
printing page 7571
printing page 7572
printing page 7573
printing page 7574
printing page 7575
printing page 7576
printing page 7577
printing page 7578
printing page 7579
printing page 7580
printing page 7581
printing page 7582

printing page 7956
printing page 7957
printing page 7958
printing page 7959
printing page 7960
printing page 7961
printing page 7962
printing page 7963
title not found
printing page 7964
printing page 7965
printing page 7966
printing page 7967
printing page 7968
printing page 7969
printing page 7970
printing page 7971
printing page 7972
printing page 7973
printing page 7974
printing page 7975
printing page 7976
printing page 7977
printing page 7978
printing page 7979
printing page 7980
printing page 7981
printing page 7982
printing page 7983
printing page 7984
printing page 7985
title not found
printing page 7986
printing page 7987
printing page 7988
printing page 7989
printing page 7990
printing page 7991
printing page 7992
printing page 7993
printing page 7994
printing page 7995
printing page 7996
printing page 7997
printing page 7998
printing page 7999
printing page 8000
titles: ['Puss in Boots: The Last Wish', 'M3GAN', 'Avatar: The Way of Water', 'The Enforcer', 'Devotion', 'Violen

##### Create dataframe to store all movie information

In [35]:
df = pd.DataFrame() 
df['title'] = ''
df['rating'] = ''          
df['genre'] = ''
df['overview'] = ''

In [38]:
# dictionary of lists 
dict = {'title': title_list, 'overview': overview_list, 'rating': ratings_list, 'genre': tag_genre_list} 
    
df = pd.DataFrame(dict)
    
df = pd.DataFrame(dict)
pd.set_option('display.max_colwidth', None)    
df

,title,overview,rating,genre
0,Puss in Boots: The Last Wish,"Puss in Boots discovers that his passion for adventure has taken its toll: He has burned through eight of his nine lives, leaving him with only one life left. Puss sets out on an epic journey to find the mythical Last Wish and restore his nine lives.",86.0,"[Animation, Action, Adventure, Comedy, Family, Fantasy]"
1,M3GAN,"A brilliant toy company roboticist uses artificial intelligence to develop M3GAN, a life-like doll programmed to emotionally bond with her newly orphaned niece. But when the doll's programming works too well, she becomes overprotective of her new friend with terrifying results.",74.0,"[Science Fiction, Horror, Comedy]"
2,Avatar: The Way of Water,"Set more than a decade after the events of the first film, learn the story of the Sully family (Jake, Neytiri, and their kids), the trouble that follows them, the lengths they go to keep each other safe, the battles they fight to stay alive, and the tragedies they endure.",77.0,"[Science Fiction, Adventure, Action]"
3,The Enforcer,"A noir thriller set in Miami, the film follows an enforcer who discovers his femme fatale boss has branched out into cyber sex trafficking, putting a young runaway he’s befriended at risk. He sacrifices everything to save the young girl from the deadly organization he’s spent his life building.",70.0,"[Action, Thriller, Drama]"
4,Devotion,The harrowing true story of two elite US Navy fighter pilots during the Korean War. Their heroic sacrifices would ultimately make them the Navy's most celebrated wingmen.,74.0,"[War, History, Drama]"
...,...,...,...,...
7995,The Librarian: Return to King Solomon's Mines,"After retrieving the Crystal Skull in Utah, Flynn Carsen receives a map in the mail with the secret location of King Solomon's Mines. When the scroll is stolen, Judson explains the power of the Key of Solomon's book and assigns Flynn to retrieve the map. The map is useless without the legend piece to decipher it, which is located in Volubilis near the Roman ruins in Morocco. Flynn heads to Casablanca to the ruins where he is chased by a group of mercenaries leaded by General Samir. They too want to find the location of King Solomon's mines. Flynn teams-up with Professor Emily Davenport working in the dig and they escape from General Samir and his men. While traveling to Gedi, they save the local Jomo from death and the trio faces a dangerous journey through the wild Africa.",62.0,"[Fantasy, Action, Adventure]"
7996,Problem Child 3,"Chapter 3 of the Problem Child trilogy features pre-teened Junior in love with a classmate that won't even notice him, but does notice three other boys who are rivals to Junior. This means war!",46.0,[Comedy]
7997,The Wing or the Thigh?,"Charles Duchemin, a well-known gourmet and publisher of a famous restaurant guide, is waging a war against fast food entrepreneur Tri- catel to save the French art of cooking. After having agreed to appear on a talk show to show his skills in naming food and wine by taste, he is confronted with two disasters: his son wants to become a clown rather than a restaurant tester and he, the famous Charles Duchemin, has lost his taste!",72.0,[Comedy]
7998,Point Blank,A nurse is forced to spring a wounded murder suspect from the hospital when the man’s brother kidnaps his pregnant wife and wants to make a trade.,60.0,"[Action, Thriller]"


In [39]:
#Put into dataset 
df.to_csv('movie_scraping.csv',index=False)